In [ ]:
# Copyright 2021 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Dataset

In this notebook we are using the `YOOCHOOSE dataset` which contains a collection of sessions from a retailer, where each session is encapsulating the click events that the user performed in that session.

The dataset is available on [Kaggle](https://www.kaggle.com/chadgostopp/recsys-challenge-2015).

- Import required libraries

In [2]:
import os
import pandas as pd
import numpy as np
import gc

import cudf
import cupy
import nvtabular as nvt
from nvtabular import ColumnSelector

In [3]:
DATA_FOLDER = "/workspace/yoochoose/"
FILENAME_PATTERN = 'yoochoose-clicks.dat'
DATA_PATH = os.path.join(DATA_FOLDER, FILENAME_PATTERN)

In [4]:
OUTPUT_FOLDER = "/workspace/yoochoose/sessions_by_day_ts"
!mkdir -p $OUTPUT_FOLDER

In [5]:
MINIMUM_SESSION_LENGTH = 2

### Loading data

In [6]:
#List of columns names to be used for CSV without the header line
HEADLESS_CSV_COLUMN_NAMES = ['session_id','timestamp', 'item_id', 'category']

In [7]:
interactions_df = cudf.read_csv(DATA_PATH, sep=',', names=HEADLESS_CSV_COLUMN_NAMES)

- Convert timestamp from datetime

In [8]:
interactions_df['event_time_dt'] = interactions_df['timestamp'].astype('datetime64[s]')
interactions_df['event_time_ts']= interactions_df['event_time_dt'].astype('int')

In [9]:
interactions_df = interactions_df.drop(['timestamp'], axis=1)

In [10]:
interactions_df.head(2)

,session_id,item_id,category,event_time_dt,event_time_ts
0,1,214536502,0,2014-04-07 10:51:09,1396867869
1,1,214536500,0,2014-04-07 10:54:09,1396868049


### Removing consecutive repeated interactions

In [11]:
print("Count with in-session repeated interactions: {}".format(len(interactions_df)))
# Sorts the dataframe by session and timestamp, to remove consective repetitions
interactions_df = interactions_df.sort_values(['session_id', 'event_time_ts'])

interactions_df['item_id_past'] = interactions_df['item_id'].shift(1).fillna(0)
interactions_df['session_id_past'] = interactions_df['session_id'].shift(1).fillna(0)
#Keeping only no consectutive repeated in session interactions
interactions_df = interactions_df[~((interactions_df['session_id'] == interactions_df['session_id_past']) & \
                 (interactions_df['item_id'] == interactions_df['item_id_past']))]
print("Count after removed in-session repeated interactions: {}".format(len(interactions_df)))

Count with in-session repeated interactions: 33003944
Count after removed in-session repeated interactions: 28971544


In [12]:
del(interactions_df['item_id_past'])
del(interactions_df['session_id_past'])

### Include the item first time seen feature (for recency calculation)

In [ ]:
items_first_ts_df = interactions_df.groupby('item_id').agg({'event_time_ts': 'min'}).reset_index().rename(columns={'event_time_ts': 'itemid_ts_first'})
interactions_merged_df = interactions_df.merge(items_first_ts_df, on=['item_id'], how='left')

In [15]:
interactions_merged_df.head()

,session_id,item_id,category,event_time_dt,event_time_ts,itemid_ts_first
0,23123,214821024,0,2014-04-02 20:43:48,1396471428,1396322465
1,23123,214587384,0,2014-04-02 20:44:51,1396471491,1396321757
2,23124,214820450,0,2014-04-01 14:54:24,1396364064,1396326153
3,23126,214820450,0,2014-04-03 14:16:28,1396534588,1396326153
4,23126,214584797,0,2014-04-03 14:20:39,1396534839,1396326293


- Check out if there is any column with missing values

In [16]:
interactions_merged_df.isnull().any()

session_id         False
item_id            False
category           False
event_time_dt      False
event_time_ts      False
itemid_ts_first    False
dtype: bool

## Temporal Features

In [ ]:
# create time features
sessionTime = ['event_time_dt']

sessionTime_hour = (
    sessionTime >> 
    #nvt.ops.LambdaOp(lambda col: cudf.to_datetime(col, unit='ms').dt.hour) >> 
    nvt.ops.LambdaOp(lambda col: col.dt.hour) >> 
    nvt.ops.Rename(name = "event_time_hour")
)
sessionTime_weekday = (
    sessionTime >> 
    #nvt.ops.LambdaOp(lambda col: cudf.to_datetime(col, unit='ms').dt.weekday) >> 
    nvt.ops.LambdaOp(lambda col: col.dt.weekday) >> 
    nvt.ops.Rename(name = "event_time_wd")
)
sessionTime_day = (
    sessionTime >> 
    nvt.ops.LambdaOp(lambda col: col.dt.day) >> 
    nvt.ops.Rename(name = "event_time_day")
)

### Temporal features

In [17]:
# create time features
sessionTime = ['event_time_dt']

sessionTime_hour = (
    sessionTime >> 
    #nvt.ops.LambdaOp(lambda col: cudf.to_datetime(col, unit='ms').dt.hour) >> 
    nvt.ops.LambdaOp(lambda col: col.dt.hour) >> 
    nvt.ops.Rename(name = "event_time_hour")
)
sessionTime_weekday = (
    sessionTime >> 
    #nvt.ops.LambdaOp(lambda col: cudf.to_datetime(col, unit='ms').dt.weekday) >> 
    nvt.ops.LambdaOp(lambda col: col.dt.weekday) >> 
    nvt.ops.Rename(name = "event_time_wd")
)
sessionTime_day = (
    sessionTime >> 
    nvt.ops.LambdaOp(lambda col: col.dt.day) >> 
    nvt.ops.Rename(name = "event_time_day")
)

In [18]:
def get_cycled_feature_value_sin(col, max_value):
    value_scaled = (col + 0.000001) / max_value
    value_sin = np.sin(2*np.pi*value_scaled)
    return value_sin

def get_cycled_feature_value_cos(col, max_value):
    value_scaled = (col + 0.000001) / max_value
    value_cos = np.cos(2*np.pi*value_scaled)
    return value_cos

In [19]:
hour_sin = sessionTime_hour >> (lambda col: get_cycled_feature_value_sin(col, 24)) >> nvt.ops.Rename(name = 'timestamp_hour_sin')
hour_cos = sessionTime_hour >> (lambda col: get_cycled_feature_value_cos(col, 24)) >> nvt.ops.Rename(name = 'timestamp_hour_cos')
weekday_sin = sessionTime_weekday >> (lambda col: get_cycled_feature_value_sin(col+1, 7)) >> nvt.ops.Rename(name = 'timestamp_wd_sin')
weekday_cos= sessionTime_weekday >> (lambda col: get_cycled_feature_value_cos(col+1, 7)) >> nvt.ops.Rename(name = 'timestamp_wd_cos')

In [20]:
cycled_features = hour_sin + hour_cos + weekday_sin + weekday_cos

- Calculate item recency 

In [21]:
# create custom op
from nvtabular.ops import Operator

class ItemRecency(Operator):
    def transform(self, columns, gdf):
        for column in columns:
            col = gdf[column]
            item_first_timestamp = gdf['itemid_ts_first']
            delta_days = (col - item_first_timestamp) / (60*60*24)
            gdf[column + "_age_days"] = delta_days * (delta_days >=0)
        return gdf
            
    def output_column_names(self, columns):
        return ColumnSelector([column + "_age_days" for column in columns])
            
    def dependencies(self):
        return ["itemid_ts_first"]

In [22]:
recency_features = ["event_time_ts"] >> ItemRecency() 
recency_features_norm = recency_features >> nvt.ops.LogOp() >> nvt.ops.Normalize() >> nvt.ops.Rename(name = "timestamp_age_days_norm")

In [23]:
recency_features_norm.selector.names

['timestamp_age_days_norm']

In [24]:
time_features = (
    sessionTime + 
    sessionTime_hour +
    sessionTime_day + 
    sessionTime_weekday +
    recency_features_norm + 
    cycled_features +
    ["event_time_ts"]
)

In [ ]:
time_features.graph

### Categorify

In [25]:
categ_features = ['item_id', 'category'] >> nvt.ops.Categorify()

### Grouping interactions into sessions

In [26]:
groupby_feats = categ_features + ['session_id']+ time_features

In [27]:
groupby_feats.selector.names

['item_id',
 'category',
 'session_id',
 'event_time_hour',
 'event_time_dt',
 'event_time_day',
 'event_time_wd',
 'timestamp_age_days_norm',
 'timestamp_hour_sin',
 'timestamp_hour_cos',
 'timestamp_wd_sin',
 'timestamp_wd_cos',
 'event_time_ts']

In [28]:
# Define Groupby Workflow
groupby_features = groupby_feats >> nvt.ops.Groupby(
    groupby_cols=["session_id"], 
    sort_cols=["event_time_ts"],
    aggs={
        "item_id": ["list", "count"],
        "category": ["list"],        
        "event_time_ts": ["list", "first"],
        "event_time_dt": ["first"],
        'event_time_day': ["list"],
        'event_time_hour': ["list"],
        'event_time_wd': ["list"],
        'timestamp_age_days_norm': ["list"],
        'timestamp_hour_sin': ["list"],
        'timestamp_hour_cos': ["list"],
        'timestamp_wd_sin': ["list"],
        'timestamp_wd_cos': ["list"],
        },
    name_sep="-")

In [29]:
groupby_features_nonlist = [x for x in groupby_features.selector if '-list' not in x]
groupby_features_nonlist

['session_id', 'event_time_dt-first', 'event_time_ts-first', 'item_id-count']

In [30]:
groupby_features_trim = ((groupby_features - groupby_features_nonlist)) >> nvt.ops.ListSlice(0,20) >> nvt.ops.Rename(postfix = '_trim')

/root/.local/lib/python3.8/site-packages/nvtabular-0.6.0+16.gd36ebd2-py3.8-linux-x86_64.egg/nvtabular/workflow/node.py:43: FutureWarning: The `["a", "b", "c"] >> ops.Operator` syntax for creating a `ColumnGroup` has been deprecated in NVTabular 21.09 and will be removed in a future version.
  warnings.warn(


In [31]:
# calculate session day index based on 'event_time_dt-first' column
day_index = ((groupby_features['event_time_dt-first'])  >> 
    nvt.ops.LambdaOp(lambda col: (col - col.min()).dt.days +1) >> 
    nvt.ops.Rename(f = lambda col: "day_index")
)

In [32]:
day_padded = day_index >> (lambda col: col.astype(str).str.pad(4,fillchar='0')) >> nvt.ops.Rename(f = lambda col: "day_padded")

In [33]:
selected_features = groupby_features[groupby_features_nonlist] + groupby_features_trim + day_padded

In [34]:
filtered_sessions = selected_features-'event_time_dt-first' >> nvt.ops.Filter(f=lambda df: df["item_id-count"] >= MINIMUM_SESSION_LENGTH)

In [35]:
filtered_sessions.selector.names

['session_id',
 'event_time_ts-first',
 'item_id-count',
 'timestamp_wd_sin-list_trim',
 'event_time_day-list_trim',
 'event_time_wd-list_trim',
 'timestamp_hour_cos-list_trim',
 'timestamp_wd_cos-list_trim',
 'category-list_trim',
 'timestamp_age_days_norm-list_trim',
 'item_id-list_trim',
 'event_time_hour-list_trim',
 'timestamp_hour_sin-list_trim',
 'event_time_ts-list_trim',
 'day_padded']

- Initialize workflow object and transform the dataset

In [36]:
workflow = nvt.Workflow(filtered_sessions)
dataset = nvt.Dataset(interactions_merged_df, cpu=False)
workflow.fit(dataset)
sessions_gdf = workflow.transform(dataset).to_ddf().compute()

/root/.local/lib/python3.8/site-packages/numba-0.54.0rc2-py3.8-linux-x86_64.egg/numba/cuda/compiler.py:865: NumbaPerformanceWarning: Grid size (1) < 2 * SM count (160) will likely result in GPU under utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


In [37]:
sessions_gdf.head(2)

,session_id,event_time_ts-first,item_id-count,timestamp_wd_sin-list_trim,event_time_day-list_trim,event_time_wd-list_trim,timestamp_hour_cos-list_trim,timestamp_wd_cos-list_trim,category-list_trim,timestamp_age_days_norm-list_trim,item_id-list_trim,event_time_hour-list_trim,timestamp_hour_sin-list_trim,event_time_ts-list_trim,day_padded
0,1,1396867869,4,"[0.7818321, 0.7818321, 0.7818321, 0.7818321]","[7, 7, 7, 7]","[0, 0, 0, 0]","[-0.8660256, -0.8660256, -0.8660256, -0.8660256]","[0.6234891, 0.6234891, 0.6234891, 0.6234891]","[1, 1, 1, 1]","[-1.3044713, -1.2986491, -1.3283161, -1.3548527]","[3559, 7340, 19523, 15499]","[10, 10, 10, 10]","[0.49999964, 0.49999964, 0.49999964, 0.49999964]","[1396867869, 1396868049, 1396868086, 1396868220]",0007
1,2,1396878997,5,"[0.7818321, 0.7818321, 0.7818321, 0.7818321, 0...","[7, 7, 7, 7, 7]","[0, 0, 0, 0, 0]","[-0.96592575, -0.96592575, -0.96592575, -0.866...","[0.6234891, 0.6234891, 0.6234891, 0.6234891, 0...","[1, 1, 1, 1, 1]","[-1.3023784, -2.2679915, -2.1763284, -2.169367...","[1968, 8039, 5178, 9410, 605]","[13, 13, 13, 14, 14]","[-0.2588193, -0.2588193, -0.2588193, -0.500000...","[1396878997, 1396879117, 1396879190, 139687923...",0007


In [38]:
categ_features.op.get_embedding_sizes(categ_features.selector)

{'item_id': (52740, 512), 'category': (336, 42)}

### Exporting session by day 

#### Saving using hive-partitioned data

In [39]:
PARTITION_COL = 'day_padded'

# Convert to a Dataset and write out hive-partitioned data to disk
nvt_output_path_tmp ='./output_nvt_tmp/'
workflow.transform(dataset).to_parquet(nvt_output_path_tmp, partition_on=[PARTITION_COL])

#### Converting to the HF4Rec dir structure and splitting dataset

- Rename columns to match with the yaml file

In [42]:
rename_cols = ['session_id', 'event_time_ts-first', 'session_size', 'timestamp_wd_sin-list', 'event_time_day-list','event_time_wd-list', 'timestamp_hour_cos-list',
               'timestamp_wd_cos-list', 'category-list', 'timestamp_age_days_norm-list', 'item_id-list', 'event_time_hour-list', 'timestamp_hour_sin-list',
               'event_time_ts-list']

In [44]:
days_folders = [f for f in sorted(os.listdir(nvt_output_path_tmp)) if f.startswith(PARTITION_COL)]
for day_folder in days_folders:
    df = cudf.read_parquet(os.path.join(nvt_output_path_tmp, day_folder))
    df = df.sort_values('event_time_ts-first')
    df.columns = rename_cols
    out_folder = os.path.join(OUTPUT_FOLDER, day_folder.replace('day_padded=', ''))
    os.makedirs(out_folder, exist_ok=True)
    df.to_parquet(os.path.join(out_folder, 'train.parquet'))
    
    random_values = cupy.random.rand(len(df))
    
    #Extracts 10% for valid and test set. Those sessions are also in the train set, but as evaluation
    #happens only for the subsequent day of training, that is not an issue, and we can keep the train set larger.
    valid_set = df[random_values <= 0.10]
    valid_set.to_parquet(os.path.join(out_folder, 'valid.parquet'))
    
    test_set = df[random_values >= 0.90]
    test_set.to_parquet(os.path.join(out_folder, 'test.parquet'))

In [45]:
df = cudf.read_parquet(os.path.join(OUTPUT_FOLDER, '0001/train.parquet'))
df.head()

,session_id,event_time_ts-first,session_size,timestamp_wd_sin-list,event_time_day-list,event_time_wd-list,timestamp_hour_cos-list,timestamp_wd_cos-list,category-list,timestamp_age_days_norm-list,item_id-list,event_time_hour-list,timestamp_hour_sin-list,event_time_ts-list
31230,351646,1396321200,2,"[0.9749277, 0.9749277]","[1, 1]","[1, 1]","[0.70710653, 0.70710653]","[-0.22252177, -0.22252177]","[1, 1]","[-3.0278223, -3.0278223]","[556, 574]","[3, 3]","[0.707107, 0.707107]","[1396321200, 1396321393]"
34633,389654,1396321200,5,"[0.9749277, 0.9749277, 0.9749277, 0.9749277, 0...","[1, 1, 1, 1, 1]","[1, 1, 1, 1, 1]","[0.70710653, 0.70710653, 0.70710653, 0.7071065...","[-0.22252177, -0.22252177, -0.22252177, -0.222...","[1, 1, 1, 1, 1]","[-3.0278223, -3.0278223, -3.026238, -3.0273852...","[90, 4519, 90, 513, 90]","[3, 3, 3, 3, 3]","[0.707107, 0.707107, 0.707107, 0.707107, 0.707...","[1396321200, 1396321248, 1396321356, 139632170..."
18660,210798,1396321213,12,"[0.9749277, 0.9749277, 0.9749277, 0.9749277, 0...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[0.70710653, 0.70710653, 0.70710653, 0.7071065...","[-0.22252177, -0.22252177, -0.22252177, -0.222...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[-3.0278223, -3.0278223, -3.0278223, -3.027822...","[6593, 5068, 12213, 15581, 11431, 1731, 11431,...","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]","[0.707107, 0.707107, 0.707107, 0.707107, 0.707...","[1396321213, 1396321268, 1396321299, 139632157..."
33364,375257,1396321213,11,"[0.9749277, 0.9749277, 0.9749277, 0.9749277, 0...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[0.70710653, 0.70710653, 0.70710653, 0.7071065...","[-0.22252177, -0.22252177, -0.22252177, -0.222...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[-3.0278223, -3.0278223, -3.0278223, -3.027822...","[4133, 4034, 5093, 1117, 4068, 1117, 4068, 111...","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]","[0.707107, 0.707107, 0.707107, 0.707107, 0.707...","[1396321213, 1396321263, 1396321416, 139632151..."
15154,171168,1396321220,2,"[0.9749277, 0.9749277]","[1, 1]","[1, 1]","[0.70710653, 0.70710653]","[-0.22252177, -0.22252177]","[1, 1]","[-3.0278223, -3.0278223]","[464, 576]","[3, 3]","[0.707107, 0.707107]","[1396321220, 1396321232]"
